## 1. Binder 驱动概述
### 1.1 概述
- Binder 驱动是 Android 专用的，但底层的驱动架构与 Linux 驱动一样。binder 驱动在以 misc 设备进行注册，作为虚拟字符设备，没有直接操作硬件，只是对设备内存的处理。主要是驱动设备的初始化(binder_init)，打开 (binder_open)，映射(binder_mmap)，数据操作(binder_ioctl)。

![image](binder1_page1.png)

### 1.2 系统调用
- 用户态的程序调用 Kernel 层驱动是需要陷入内核态，进行系统调用(syscall)，比如打开 Binder 驱动方法的调用链为： open-> __ open() -> binder_open()。 open() 为用户空间的方法，__open() 便是系统调用中相应的处理方法，通过查找，对应调用到内核 binder 驱动的 binder_open() 方法，至于其他的从用户态陷入内核态的流程也基本一致。

![image](binder1_page2.png)

---

## 2. Binder 核心方法
### 2.1 binder_init
- 主要工作是为了注册 misc 设备
- debugfs_create_dir 是指在 debugfs 文件系统中创建一个目录，返回值是指向 dentry 的指针。当 kernel 中禁用 debugfs 的话，返回值是 -%ENODEV。默认是禁用的。如果需要打开，在目录 /kernel/arch/arm64/configs/ 下找到目标 defconfig 文件中添加一行 CONFIG_DEBUG_FS=y，再重新编译版本，即可打开 debug_fs。

```
static int __init binder_init(void)
{
    int ret;
    // 创建名为 binder 的工作队列
    binder_deferred_workqueue = create_singlethread_workqueue("binder");
    ...

    binder_debugfs_dir_entry_root = debugfs_create_dir("binder", NULL);
    if (binder_debugfs_dir_entry_root)
        binder_debugfs_dir_entry_proc = debugfs_create_dir("proc",
                         binder_debugfs_dir_entry_root);

     // 注册 misc 设备【见小节2.1.1】
    ret = misc_register(&binder_miscdev);
    if (binder_debugfs_dir_entry_root) {
        ... // 在 debugfs 文件系统中创建一系列的文件
    }
    return ret;
}
```

#### 2.1.1 misc_register
- 注册 misc 设备，miscdevice 结构体，便是前面注册 misc 设备时传递进去的参数

```
static struct miscdevice binder_miscdev = {
    .minor = MISC_DYNAMIC_MINOR, // 次设备号 动态分配
    .name = "binder",            // 设备名
    .fops = &binder_fops         // 设备的文件操作结构，这是 file_operations 结构
};
```

- file_operations 结构体, 指定相应文件操作的方法
```
static const struct file_operations binder_fops = {
    .owner = THIS_MODULE,
    .poll = binder_poll,
    .unlocked_ioctl = binder_ioctl,
    .compat_ioctl = binder_ioctl,
    .mmap = binder_mmap,
    .open = binder_open,
    .flush = binder_flush,
    .release = binder_release,
};
```

---

### 2.2 binder_open
- 打开 binder 驱动设备

```
static int binder_open(struct inode *nodp, struct file *filp)
{
    struct binder_proc *proc;   // binder 进程 【见附录3.1】

    proc = kzalloc(sizeof(*proc), GFP_KERNEL); // 为 binder_proc 结构体在分配 kernel 内存空间
    if (proc == NULL)
        return -ENOMEM;
    get_task_struct(current);
    proc->tsk = current;          // 将当前线程的 task 保存到 binder 进程的 tsk
    INIT_LIST_HEAD(&proc->todo);  // 初始化 todo 列表
    init_waitqueue_head(&proc->wait); // 初始化 wait 队列
    proc->default_priority = task_nice(current);  // 将当前进程的 nice 值转换为进程优先级

    binder_lock(__func__);   // 同步锁，因为binder支持多线程访问
    binder_stats_created(BINDER_STAT_PROC); // BINDER_PROC 对象创建数加1
    hlist_add_head(&proc->proc_node, &binder_procs); // 将 proc_node 节点添加到 binder_procs 为表头的队列
    proc->pid = current->group_leader->pid;
    INIT_LIST_HEAD(&proc->delivered_death); // 初始化已分发的死亡通知列表
    filp->private_data = proc;        // file 文件指针的 private_data 变量指向 binder_proc 数据
    binder_unlock(__func__);          // 释放同步锁

    return 0;
}
```

- 创建 binder_proc 对象，并把当前进程等信息保存到 binder_proc 对象，该对象管理 IPC 所需的各种信息并拥有其他结构体的根结构体；再把 binder_proc 对象保存到文件指针 filp，以及把 binder_proc 加入到全局链表 binder_procs。
- Binder 驱动中通过 static HLIST_HEAD(binder_procs);，创建了全局的哈希链表 binder_procs，用于保存所有的 binder_proc 队列，每次新创建的 binder_proc 对象都会加入 binder_procs 链表中。
![image](binder1_page3.png)

---

### 2.3 binder_mmap
- 主要功能：首先在内核虚拟地址空间，申请一块与用户虚拟内存相同大小的内存；然后再申请1个 page 大小的物理内存，再将同一块物理内存分别映射到内核虚拟地址空间和用户虚拟内存空间，从而实现了用户空间的 Buffer 和内核空间的 Buffer 同步操作的功能。

```
static int binder_mmap(struct file *filp, struct vm_area_struct *vma)
{
    int ret;
    struct vm_struct *area; //内核虚拟空间
    struct binder_proc *proc = filp->private_data;
    const char *failure_string;
    struct binder_buffer *buffer;  //【见附录3.9】

    if (proc->tsk != current)
        return -EINVAL;

    if ((vma->vm_end - vma->vm_start) > SZ_4M)
        vma->vm_end = vma->vm_start + SZ_4M;  //保证映射内存大小不超过4M

    mutex_lock(&binder_mmap_lock);  //同步锁
    //采用IOREMAP方式，分配一个连续的内核虚拟空间，与进程虚拟空间大小一致
    area = get_vm_area(vma->vm_end - vma->vm_start, VM_IOREMAP);
    if (area == NULL) {
        ret = -ENOMEM;
        failure_string = "get_vm_area";
        goto err_get_vm_area_failed;
    }
    proc->buffer = area->addr; //指向内核虚拟空间的地址
    //地址偏移量 = 用户虚拟地址空间 - 内核虚拟地址空间
    proc->user_buffer_offset = vma->vm_start - (uintptr_t)proc->buffer;
    mutex_unlock(&binder_mmap_lock); //释放锁

    ...
    //分配物理页的指针数组，数组大小为vma的等效page个数；
    proc->pages = kzalloc(sizeof(proc->pages[0]) * ((vma->vm_end - vma->vm_start) / PAGE_SIZE), GFP_KERNEL);
    if (proc->pages == NULL) {
        ret = -ENOMEM;
        failure_string = "alloc page array";
        goto err_alloc_pages_failed;
    }
    proc->buffer_size = vma->vm_end - vma->vm_start;

    vma->vm_ops = &binder_vm_ops;
    vma->vm_private_data = proc;

    //分配物理页面，同时映射到内核空间和进程空间，先分配1个物理页 【见小节2.3.1】
    if (binder_update_page_range(proc, 1, proc->buffer, proc->buffer + PAGE_SIZE, vma)) {
        ret = -ENOMEM;
        failure_string = "alloc small buf";
        goto err_alloc_small_buf_failed;
    }
    buffer = proc->buffer; //binder_buffer对象 指向proc的buffer地址
    INIT_LIST_HEAD(&proc->buffers); //创建进程的buffers链表头
    list_add(&buffer->entry, &proc->buffers); //将binder_buffer地址 加入到所属进程的buffers队列
    buffer->free = 1;
    //将空闲buffer放入proc->free_buffers中
    binder_insert_free_buffer(proc, buffer);
    //异步可用空间大小为buffer总大小的一半。
    proc->free_async_space = proc->buffer_size / 2;
    barrier();
    proc->files = get_files_struct(current);
    proc->vma = vma;
    proc->vma_vm_mm = vma->vm_mm;
    return 0;

    ...// 错误flags跳转处，free释放内存之类的操作
    return ret;
}
```

- binder_mmap 通过加锁，保证一次只有一个进程分配内存，保证多进程间的并发访问。其中 user_buffer_offset 是虚拟进程地址与虚拟内核地址的差值(该值为负数)。也就是说同一物理地址，当内核地址为 kernel_addr，则进程地址为 proc_addr = kernel_addr + user_buffer_offset。

#### 2.3.1 binder_update_page_range

```
static int binder_update_page_range(struct binder_proc *proc, int allocate,
            void *start, void *end,
            struct vm_area_struct *vma)
{
  void *page_addr;
  unsigned long user_page_addr;
  struct page **page;
  struct mm_struct *mm; // 内存结构体

  if (vma)
  		mm = NULL; // binder_mmap 过程 vma 不为空，其他情况都为空
  	else
  		mm = get_task_mm(proc->tsk); // 获取 mm 结构体
      
  if (mm) {
    down_write(&mm->mmap_sem); // 获取 mm_struct 的写信号量
    vma = proc->vma;
  }

  // 此处 allocate 为 1，代表分配过程。如果为 0 则代表释放过程
  if (allocate == 0)
    goto free_range;

  for (page_addr = start; page_addr < end; page_addr += PAGE_SIZE) {
    int ret;
    page = &proc->pages[(page_addr - proc->buffer) / PAGE_SIZE];
    // 分配一个page的物理内存
    *page = alloc_page(GFP_KERNEL | __GFP_HIGHMEM | __GFP_ZERO);
    
    // 物理空间映射到虚拟内核空间
    ret = map_kernel_range_noflush((unsigned long)page_addr,
          PAGE_SIZE, PAGE_KERNEL, page);
    flush_cache_vmap((unsigned long)page_addr, (unsigned long)page_addr + PAGE_SIZE);
  
    user_page_addr = (uintptr_t)page_addr + proc->user_buffer_offset;
    // 物理空间映射到虚拟进程空间
    ret = vm_insert_page(vma, user_page_addr, page[0]);
  }
  
  if (mm) {
    up_write(&mm->mmap_sem); // 释放内存的写信号量
    mmput(mm); // 减少mm->mm_users计数
  }
  return 0;

free_range:
  ... // 释放内存的流程
  
  return -ENOMEM;
}
```

![image](binder1_page4.png)

- 主要工作如下：
  - binder_update_page_range 主要完成工作：分配物理空间，将物理空间映射到内核空间，将物理空间映射到进程空间。另外，不同参数下该方法也可以释放物理页面。

- binder_update_page_range 的调用时机：
  - binder_mmap: 用于分配内存，分配大小为 1 page, vma 不为空
  - binder_alloc_buf：用于分配内存，vma 为空
  - binder_free_buf: 用于释放内存，vma 为空
  - binder_delete_free_buffer：同样用于释放内存，vma 为空
  
- mm_struct 结构体，定义在 mm_types.h 文件：

```
struct mm_struct {
  struct vm_area_struct *mmap;   // VMA 列表
  struct rb_root mm_rb;
  pgd_t * pgd;
  atomic_t mm_users;             // 使用该内存的进程个数
  atomic_t mm_count;             // 结构体 mm_struct 的引用个数
  struct rw_semaphore mmap_sem;  // 读写信号量，用于同步
  unsigned long flags; 
  ...
};
```

#### 2.3.2 binder_alloc_buf
- 通过 binder_alloc_buf() 方法来分配 binder_buffer 结构体, 只有在 binder_transaction 过程才需要分配 buffer。

```
static struct binder_buffer *binder_alloc_buf(struct binder_proc *proc,
                          size_t data_size, size_t offsets_size, int is_async)
{
    struct rb_node *n = proc->free_buffers.rb_node;
    struct binder_buffer *buffer;
    size_t buffer_size;
    struct rb_node *best_fit = NULL;
    void *has_page_addr;
    void *end_page_addr;
    size_t size;
    if (proc->vma == NULL) {
        return NULL; //虚拟地址空间为空，直接返回
    }
    size = ALIGN(data_size, sizeof(void *)) + ALIGN(offsets_size, sizeof(void *));
    if (size < data_size || size < offsets_size) {
        return NULL; //非法的size
    }
    if (is_async && proc->free_async_space < size + sizeof(struct binder_buffer)) {
        return NULL; // 剩余可用的异步空间，小于所需的大小
    }
    while (n) {  //从binder_buffer的红黑树中查找大小相等的buffer块
        buffer = rb_entry(n, struct binder_buffer, rb_node);
        buffer_size = binder_buffer_size(proc, buffer);
        if (size < buffer_size) {
            best_fit = n;
            n = n->rb_left;
        } else if (size > buffer_size)
            n = n->rb_right;
        else {
            best_fit = n;
            break;
        }
    }
    if (best_fit == NULL) {
        return NULL; //内存分配失败，地址空间为空
    }
    if (n == NULL) {
        buffer = rb_entry(best_fit, struct binder_buffer, rb_node);
        buffer_size = binder_buffer_size(proc, buffer);
    }

    has_page_addr =(void *)(((uintptr_t)buffer->data + buffer_size) & PAGE_MASK);
    if (n == NULL) {
        if (size + sizeof(struct binder_buffer) + 4 >= buffer_size)
            buffer_size = size;
        else
            buffer_size = size + sizeof(struct binder_buffer);
    }
    end_page_addr =     (void *)PAGE_ALIGN((uintptr_t)buffer->data + buffer_size);
    if (end_page_addr > has_page_addr)
        end_page_addr = has_page_addr;
    if (binder_update_page_range(proc, 1,
        (void *)PAGE_ALIGN((uintptr_t)buffer->data), end_page_addr, NULL))
        return NULL;
    rb_erase(best_fit, &proc->free_buffers);
    buffer->free = 0;
    binder_insert_allocated_buffer(proc, buffer);
    if (buffer_size != size) {
        struct binder_buffer *new_buffer = (void *)buffer->data + size;
        list_add(&new_buffer->entry, &buffer->entry);
        new_buffer->free = 1;
        binder_insert_free_buffer(proc, new_buffer);
    }

    buffer->data_size = data_size;
    buffer->offsets_size = offsets_size;
    buffer->async_transaction = is_async;
    if (is_async) {
        proc->free_async_space -= size + sizeof(struct binder_buffer);
    }
    return buffer;
}
```

- 这里介绍的 binder_alloc_buf 是内存分配函数。除此之外，还有内存释放相关方法：
  - binder_free_buf
  - binder_delete_free_buffer
  - binder_transaction_buffer_release
  
---

### 2.4 binder_ioctl
- binder_ioctl() 函数负责在两个进程间收发 IPC 数据和 IPC reply 数据
  - (1) 文件描述符，是通过 open() 方法打开 Binder Driver 后返回值
  - (2) ioctl 命令和数据类型是一体的，不同的命令对应不同的数据类型
  
![image](binder1_page5.png)

```
static long binder_ioctl(struct file *filp, unsigned int cmd, unsigned long arg)
{
    int ret;
    struct binder_proc *proc = filp->private_data;
    struct binder_thread *thread;  // binder线程
    unsigned int size = _IOC_SIZE(cmd);
    void __user *ubuf = (void __user *)arg;
    //进入休眠状态，直到中断唤醒
    ret = wait_event_interruptible(binder_user_error_wait, binder_stop_on_user_error < 2);
    if (ret)
        goto err_unlocked;

    binder_lock(__func__);
    //获取binder_thread【见2.4.1】
    thread = binder_get_thread(proc);
    if (thread == NULL) {
        ret = -ENOMEM;
        goto err;
    }

    switch (cmd) {
    case BINDER_WRITE_READ:  //进行binder的读写操作
        ret = binder_ioctl_write_read(filp, cmd, arg, thread); //【见2.4.2】
        if (ret)
            goto err;
        break;
    case BINDER_SET_MAX_THREADS: //设置binder最大支持的线程数
        if (copy_from_user(&proc->max_threads, ubuf, sizeof(proc->max_threads))) {
            ret = -EINVAL;
            goto err;
        }
        break;
    case BINDER_SET_CONTEXT_MGR: //成为binder的上下文管理者，也就是ServiceManager成为守护进程
        ret = binder_ioctl_set_ctx_mgr(filp);
        if (ret)
            goto err;
        break;
    case BINDER_THREAD_EXIT:   //当binder线程退出，释放binder线程
        binder_free_thread(proc, thread);
        thread = NULL;
        break;
    case BINDER_VERSION: {  //获取binder的版本号
        struct binder_version __user *ver = ubuf;

        if (size != sizeof(struct binder_version)) {
            ret = -EINVAL;
            goto err;
        }
        if (put_user(BINDER_CURRENT_PROTOCOL_VERSION,
                 &ver->protocol_version)) {
            ret = -EINVAL;
            goto err;
        }
        break;
    }
    default:
        ret = -EINVAL;
        goto err;
    }
    ret = 0;
err:
    if (thread)
        thread->looper &= ~BINDER_LOOPER_STATE_NEED_RETURN;
    binder_unlock(__func__);
    wait_event_interruptible(binder_user_error_wait, binder_stop_on_user_error < 2);

err_unlocked:
    trace_binder_ioctl_done(ret);
    return ret;
}
```

#### 2.4.1 binder_get_thread
- 从 binder_proc 中查找 binder_thread, 如果当前线程已经加入到 proc 的线程队列则直接返回，如果不存在则创建 binder_thread，并将当前线程添加到当前的 proc

```
static struct binder_thread *binder_get_thread(struct binder_proc *proc)
{
    struct binder_thread *thread = NULL;
    struct rb_node *parent = NULL;
    struct rb_node **p = &proc->threads.rb_node;
    while (*p) {  //根据当前进程的pid，从binder_proc中查找相应的binder_thread
        parent = *p;
        thread = rb_entry(parent, struct binder_thread, rb_node);
        if (current->pid < thread->pid)
            p = &(*p)->rb_left;
        else if (current->pid > thread->pid)
            p = &(*p)->rb_right;
        else
            break;
    }
    if (*p == NULL) {
        thread = kzalloc(sizeof(*thread), GFP_KERNEL); //新建binder_thread结构体
        if (thread == NULL)
            return NULL;
        binder_stats_created(BINDER_STAT_THREAD);
        thread->proc = proc;
        thread->pid = current->pid;  //保存当前进程(线程)的pid
        init_waitqueue_head(&thread->wait);
        INIT_LIST_HEAD(&thread->todo);
        rb_link_node(&thread->rb_node, parent, p);
        rb_insert_color(&thread->rb_node, &proc->threads);
        thread->looper |= BINDER_LOOPER_STATE_NEED_RETURN;
        thread->return_error = BR_OK;
        thread->return_error2 = BR_OK;
    }
    return thread;
}
```

#### 2.4.2 binder_ioctl_write_read
- 对于 ioctl() 方法中，传递进来的命令是 cmd = BINDER_WRITE_READ 时执行该方法，arg 是一个 binder_write_read 结构体

```
static int binder_ioctl_write_read(struct file *filp,
                unsigned int cmd, unsigned long arg,
                struct binder_thread *thread)
{
    int ret = 0;
    struct binder_proc *proc = filp->private_data;
    unsigned int size = _IOC_SIZE(cmd);
    void __user *ubuf = (void __user *)arg;
    struct binder_write_read bwr;

    if (size != sizeof(struct binder_write_read)) {
        ret = -EINVAL;
        goto out;
    }
    if (copy_from_user(&bwr, ubuf, sizeof(bwr))) { //把用户空间数据ubuf拷贝到bwr
        ret = -EFAULT;
        goto out;
    }

    if (bwr.write_size > 0) {
        //当写缓存中有数据，则执行binder写操作
        ret = binder_thread_write(proc, thread,
                      bwr.write_buffer, bwr.write_size, &bwr.write_consumed);
        trace_binder_write_done(ret);
        if (ret < 0) { //当写失败，再将bwr数据写回用户空间，并返回
            bwr.read_consumed = 0;
            if (copy_to_user(ubuf, &bwr, sizeof(bwr)))
                ret = -EFAULT;
            goto out;
        }
    }
    if (bwr.read_size > 0) {
        //当读缓存中有数据，则执行binder读操作
        ret = binder_thread_read(proc, thread,
                      bwr.read_buffer, bwr.read_size, &bwr.read_consumed,
                      filp->f_flags & O_NONBLOCK);
        trace_binder_read_done(ret);
        if (!list_empty(&proc->todo))
            wake_up_interruptible(&proc->wait); //唤醒等待状态的线程
        if (ret < 0) { //当读失败，再将bwr数据写回用户空间，并返回
            if (copy_to_user(ubuf, &bwr, sizeof(bwr)))
                ret = -EFAULT;
            goto out;
        }
    }

    if (copy_to_user(ubuf, &bwr, sizeof(bwr))) { //将内核数据bwr拷贝到用户空间ubuf
        ret = -EFAULT;
        goto out;
    }
out:
    return ret;
}
```

- 对于binder_ioctl_write_read的流程图，如下：
![image](binder1_page6.png)

- 流程：
  - 首先，把用户空间数据 ubuf 拷贝到内核空间 bwr
  - 当 bwr 写缓存有数据，则执行 binder_thread_write；当写失败则将 bwr 数据写回用户空间并退出
  - 当 bwr 读缓存有数据，则执行 binder_thread_read；当读失败则再将 bwr 数据写回用户空间并退出
  - 最后，把内核数据 bwr 拷贝到用户空间 ubuf

---

### 2.4 Command 使用场景
- ioctl 命令常见命令的使用场景，其中 BINDER_WRITE_READ 最为频繁
  - BINDER_WRITE_READ
    - Binder 读写交互场景，IPC.talkWithDriver
  - BINDER_SET_CONTEXT_MGR
    - servicemanager 进程成为上下文管理者，binder_become_context_manager()
  - BINDER_SET_MAX_THREADS
    - 初始化 ProcessState 对象，open_driver()
    - 主动调整参数，ProcessState.setThreadPoolMaxThreadCount()
  - BINDER_VERSION
    - 初始化 ProcessState 对象，open_driver()

---

### 2.5 小节
- binder_init：初始化字符设备
- binder_open：打开驱动设备，过程需要持有 binder_main_lock 同步锁
- binder_mmap：申请内存空间，该过程需要持有 binder_mmap_lock 同步锁
- binder_ioctl：执行相应的 ioctl 操作，该过程需要持有 binder_main_lock 同步锁
  - 当处于 binder_thread_read 过程，read_buffer 无数据则释放同步锁，并处于 wait_event_freezable 过程，等有数据到来则唤醒并尝试持有同步锁

---